<a href="https://colab.research.google.com/github/ArthurCBx/PyTorch-DeepLearning-Udemy/blob/main/07_Experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 PyTorch Experiment Tracking

Machine learning is very experimental.

In order to figure out which experiments are worth pursuing, that's where **experiment tracking** comes in, it helps you to figure out what doesn't work so you can figure out what **does work**

In this notebook, we're going to see an example of programmatically tracking experiments.



In [ ]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

In [ ]:
try:
  from going_modular.going_modular import data_setup, engine
except:
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup, engine

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
def set_seeds(seed: int=42):
  """Sets random seeds for torch operations

  Args:
    Seed(int,optional): Random seed to set. Defaults to 42.
  """
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [ ]:
set_seeds()

## 1. Get data

Want to get pizza, steak sushi images.

So we can run experiments building FoodVision Mini and see wich model performs best.

In [ ]:
import os
from pathlib import Path
import zipfile
import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
  """Downloads a zipped dataset from source and unzips to destination."""
  data_path = Path("data/")
  image_path = data_path / destination

  if image_path.is_dir():
    print(f"[INFO] {image_path} directory already exists, skipping download.")
  else:
    print(f"[INFO] Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

  target_file = Path(source).name
  with open(data_path / target_file, "wb") as f:
    request = requests.get(source)
    print(f"[INFO] Downloading {target_file} from {source}...")
    f.write(request.content)

  with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
    print(f"[INFO] Unzipping {target_file} data...")
    zip_ref.extractall(image_path)

  if remove_source:
    os.remove(data_path / target_file)

  return image_path

In [ ]:
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

## 2. Create Datasets and DataLoaders

### 2.1 Create DataLoaders with manual transforms

The goal with transforms is to ensure your custom data is formatted in a reproducible way as well as a way that will suit pretrained models.

In [ ]:
# Setup directories
train_dir = image_path  / "train"
test_dir = image_path / "test"

train_dir, test_dir

In [ ]:
# Setup ImageNet normalization levels
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                                 std=[0.229,0.224,0.225])

# Create transform pipeline manually
manual_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

from going_modular.going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

### 2.2 Create DataLoader using automatically created transforms

The same principle applies for automatic transforms: we want our custom data in the same format as a pretrained model was trained on.

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transform = weights.transforms()

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=auto_transform,
                                                                               batch_size=32)

## 3. Getting a pretrained model, freeze the base layers and change the classifier head

In [ ]:
# Note: this is how a pretrained model would be created prior to torchvision v0.13
#model = torchvision.models.efficientnet_b0(pretrained=True).to(device)

# Download the pretrained weighs for EfiicientNet_B0
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Setup the model with the pretrained weights and send it to the target device
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model

In [ ]:
# Freeze all base layers by setting their requires_grad attribute to False
for param in model.features.parameters():
  param.requires_grad = False

In [ ]:
set_seeds()
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=len(class_names)
                    ).to(device)
)

In [ ]:
!pip install -q torchinfo

In [ ]:
from torchinfo import summary

summary(model=model,
        input_size=(32,3,224,224),
        verbose=0,
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])

## 4. Train a single model and track results

In [ ]:
from torch import nn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.001)

To track experiments, we're going to use TensorBoard: https://www.tensorflow.org/tensorboard/

And to interact with Tensorboard, we can use PyTorch's SummaryWritter - https://pytorch.org/docs/stable/tensorboard.html

In [ ]:
# Setup a SummaryWriter
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
writer

In [ ]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from going_modular.going_modular.engine import train_step,test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc},
                           global_step=epoch)

        writer.add_graph(model=model,
                         input_to_model=torch.randn(32,3,224,224).to(device))

    # Close the writer
    writer.close()
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

In [ ]:
# Train model
# Note: not using engine.train(), since we updated the train() function above
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

## 5. View our model's results with TensorBoard

There are a few ways to view TensorBoard results, see them here: https://www.learnpytorch.io/07_pytorch_experiment_tracking/#5-view-our-models-results-in-tensorboard

In [ ]:
# Let's view our experiments from within the notebook
%load_ext tensorboard
%tensorboard --logdir runs

## 6. Create a function to prepare a `SummaryWriter()` instance

By default our `SummaryWriter()` class saves to `log_dir`.

How about if we wanted to save different experiments to different folders?

In essence, one experiment = one folder.

For example, we'd like to track:
* Experiment date/timestamp
* Experiment name
* Model name
* Extra - is there anything else that should be tracked?

Let's create a function to cerate a `SummaryWriter()` instance to take all of these things into account.

So ideally we end up tracking experiments to a directory:

`runs/YYYY-MM-DD/experiment_name/model_name/extra`

In [ ]:
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str = None):
  """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance tracking to a specific directory."""

  from datetime import datetime
  import os

  # Get timestamp of current date in reverse order
  timestamp = datetime.now().strftime("%Y-%m-%d")
  if extra:
    # Create log directory path
    log_dir = os.path.join("runs",timestamp,experiment_name,model_name,extra)
  else:
    log_dir = os.path.join("runs",timestamp,experiment_name,model_name)

  print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
  return SummaryWriter(log_dir=log_dir)


In [ ]:
example_writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb0",
                               extra="5_epochs")

example_writer

### 6.1 Update the `train()` function to include a `writer` parameter

In [ ]:
# Update train() to use create_writer()

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from going_modular.going_modular.engine import train_step,test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        if writer:
          writer.add_scalars(main_tag="Loss",
                             tag_scalar_dict={"train_loss": train_loss,
                                              "test_loss": test_loss},
                             global_step=epoch)

          writer.add_scalars(main_tag="Accuracy",
                             tag_scalar_dict={"train_acc": train_acc,
                                              "test_acc": test_acc},
                             global_step=epoch)

          writer.add_graph(model=model,
                           input_to_model=torch.randn(32,3,224,224).to(device))

          # Close the writer
          writer.close()
        else:
          pass
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

## 7. Setting up a series of modelling experiments

* Setup 2x modelling experiments with effmetn0, pizza, steak, sushi data and train one model for 5 epochs and another model for 10 epochs

### 7.1 What kind of experiments should you run?

The number of machine learning experiments you can run, is like the number of different models you can build... almost limitless.

However, you can't test everything...

So, what should you test?
* Change the number of epochs
* Change the number of hidden units
* Change the amount of data (right now we're using 10% of the Food101 dataset for pizza, steak sushi)
* Change the learning rate
* Try different kinds of data augmentation
* Choose a different model architecture

This is why transfer learning is so powerful, because, it's a working model that you can apply to your own problem.

### 7.2 What experiments are we going to run?

We're going to turn three dials:
1. Model size - EffnetB0 vs EffnetB2
2. Dataset size - 10% of pizza, steak, sushi images vs 20% (generally more data = better results)
3. Training time - 5 epochs vs 10 epochs

To begin, we're still keeping relatively small so that our experiments run quickly.

**Our goal:** a model that os well performing but still small enough to run on a mobile device or browser, so FoodVision Mini can come to life.

If you had infinite compute + time, you should basically always choose the biggest model and biggest dataset you can. http://www.incompleteideas.net/IncIdeas/BitterLesson.html

### 7.3 Download different datasets

We want two datasets:
1. Pizza, steak, sushi 10% - https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip
2. Pizza, steak, sushi 20% - https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip



In [ ]:
# Download 10% and 20% datasets
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

### 7.4 Transform Datasets and Create DataLoaders

we'll need to transform our data so that they have the same format as the data our pretrained model was trained on.

In [ ]:
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup the test directory used for both models
test_dir = data_10_percent_path / "test"

train_dir_10_percent,train_dir_20_percent, test_dir

In [ ]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])


In [ ]:
BATCH_SIZE=32

# Create 10% training and test DataLoaders
train_dataloader_10_percent,test_dataloader,class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
                                                                                        test_dir=test_dir,
                                                                                        transform=simple_transform,
                                                                                        batch_size=BATCH_SIZE)

# Create 20% training and test DataLoaders
train_dataloader_20_percent,test_dataloader,class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                        test_dir=test_dir,
                                                                                        transform=simple_transform,
                                                                                        batch_size=BATCH_SIZE)

print(f"Number of batches of size {BATCH_SIZE} in 10% train data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20% train data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 10% test data: {len(test_dataloader)}")
print(f"Class names: {class_names}")

### 7.5 Create feature extractor models

We want two functions:
1. Creates a `torchvision.models.efficientnet_b0()` feature extractor with a frozen backbone/base layers and a custom classifier head.
2. Creates a `torchvision.models.efficientnet_b2()` feature extractor with a frozen backbone/base layers and a custom classifier head.

In [ ]:
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

In [ ]:
import torchvision
from torch import nn

OUT_FEATURES = len(class_names)

def create_effnetb0():
  # Get the weights and setup a model
  weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
  model = torchvision.models.efficientnet_b0(weights=weights).to(device)

  # Freeze the base model layers
  for param in model.features.parameters():
    param.requires_grad = False

  # Change the classifier head
  set_seeds()
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.2, inplace=True),
      nn.Linear(in_features=1280,out_features=OUT_FEATURES)
  ).to(device)

  # Give the model a name
  model.name = "effnetb0"
  print(f"[INFO] Created new {model.name} model...")
  return model

def create_effnetb2():
  # Get the weights and setup a model
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  model = torchvision.models.efficientnet_b2(weights=weights).to(device)

  # Freeze the base model layers
  for param in model.features.parameters():
    param.requires_grad = False

  # Change the classifier head
  set_seeds()
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408,out_features=OUT_FEATURES)
  ).to(device)

  # Give the model a name
  model.name = "effnetb2"
  print(f"[INFO] Created new {model.name} model...")
  return model

In [ ]:
created_model_test_effnetb0 = create_effnetb0()
created_model_test_effnetb2 = create_effnetb2()

summary(model=created_model_test_effnetb0,
        input_size=(32,3,224,224),
        verbose=0,
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])




In [ ]:
summary(model=created_model_test_effnetb2,
        input_size=(32,3,224,224),
        verbose=0,
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])

### 7.6 Create experiments and set up training code

In [ ]:
# Create epoch list
num_epochs = [5,10]

# Create models list
models = ["effnetb0", "effnetb2"]

# Create a DataLoaders dictionary
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}


In [ ]:
%%time
from going_modular.going_modular.utils import save_model

set_seeds()

# Keep track of experiment numbers
experiment_number = 0

# Loop through each DataLoader
for dataloader_name, train_dataloader in train_dataloaders.items():
  #Loop through the epochs
  for epoch in num_epochs:
    # Loop through each model name and create a new model instance
    for model_name in models:

      #Print out info
      experiment_number += 1
      print(f"[INFO] Experiment number: {experiment_number}")
      print(f"[INFO] Model : {model_name}")
      print(f"[INFO] DataLoader : {dataloader_name}")
      print(f"[INFO] Number of epochs: {epoch}")

      # Select and create the model
      if model_name == "efnetb0":
        model = create_effnetb0()
      else:
        model = create_effnetb2()

      # Create a new loss and optimizer for every model
      loss_fn = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)

      # Train target model with target dataloader
      train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            epochs=epoch,
            device=device,
            writer=create_writer(experiment_name=dataloader_name,
                                 model_name=model_name,
                                 extra=f"{epoch}_epochs"))

      # Save the model to file so we can import it later if need be
      save_filepath = f"07_{model_name}_{dataloader_name}_{epoch}_epochs.pth"
      save_model(model=model,
                 target_dir="models",
                 model_name=save_filepath)
      print("-"*50 + "\n")

## 8. View experiments in TensorBoard

We've experimented, now let's visualize

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# Upload the results to TensorBoard.dev
!tensorboard dev upload --logdir runs \
    --name "07. PyTorch Experiment Tracking: FoodVision Mini model results" \
    --description "Comparing results of different model size, training data amount and training time."

## 9. Load in the best performing model and make predictions with it

This is our best model filepath: `models/07_effnetb2_data_20_percent_10_epochs.pth`

In [ ]:
best_model_path = "models/07_effnetb2_data_20_percent_10_epochs.pth"

best_model = create_effnetb2()
best_model.load_state_dict(torch.load(best_model_path))

Our goal: create a FoodVision Mini model that performs well enough and is able to run on a mobile device/web browser

In [ ]:
# Check the model file size
from pathlib import Path

# Get the model size in bytes then convert it to megabytes
effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
print(f"[INFO] EffNetB2 model size: {effnetb2_model_size} MB")

In [ ]:
# Import function to make predictions on images
from going_modular.going_modular.predictions import pred_and_plot_image

# Get a random list of 3 image path names from the test dataset
import random
num_images_to_plot = 3
test_image_path_list = list(Path(data_20_percent_path / "test").glob("*/*.jpg"))
test_image_path_sample = random.sample(test_image_path_list,k=num_images_to_plot)

# Make and plot predictions on the test dataset
for image_path in test_image_path_sample:
  pred_and_plot_image(model=best_model,
                      class_names=class_names,
                      image_path=image_path,
                      image_size=(224,224))

### 9.1 Predict on a custom image

In [ ]:
custom_pizza_path = Path("data/custom_image/pizza_custom.png")
pred_and_plot_image(model=best_model,
                    class_names=class_names,
                    image_path="data/custom_image/pizza_custom.png",
                    )

## Exercises

In [ ]:
weights = torchvision.models.EfficientNet_B7_Weights.DEFAULT
effnetb7_5_epochs_20_percent = torchvision.models.efficientnet_b7(weights=weights).to(device)
effnetb7_10_epochs_20_percent =torchvision.models.efficientnet_b7(weights=weights).to(device)
models = [effnetb7_5_epochs_20_percent,effnetb7_10_epochs_20_percent]
effnetb7_10_epochs_20_percent

for model in models:
  for param in model.features.parameters():
    param.requires_grad = False

  model.classifier = nn.Sequential(
      nn.Dropout(p=0.5, inplace=True),
      nn.Linear(in_features=2560,out_features=len(class_names))
  )
loss_fn_5_epochs = nn.CrossEntropyLoss()
loss_fn_10_epochs = nn.CrossEntropyLoss()

optimizer_5_epochs = torch.optim.Adam(params=effnetb7_5_epochs_20_percent.parameters(),lr=0.001)
optimizer_10_epochs = torch.optim.Adam(params=effnetb7_10_epochs_20_percent.parameters(),lr=0.001)


train(model=effnetb7_5_epochs_20_percent,
      train_dataloader=train_dataloader_20_percent,
      test_dataloader=test_dataloader,
      optimizer=optimizer_5_epochs,
      loss_fn=loss_fn_5_epochs,
      epochs=5,
      device=device,
      writer=create_writer(experiment_name="5_epochs",
                           model_name="effnetb7_5_epochs_20_percent"))

train(model=effnetb7_5_epochs_20_percent,
      train_dataloader=train_dataloader_20_percent,
      test_dataloader=test_dataloader,
      optimizer=optimizer_10_epochs,
      loss_fn=loss_fn_10_epochs,
      epochs=10,
      device=device,
      writer=create_writer(experiment_name="10_epochs",
model_name="effnetb7_10_epochs_20_percent"))


In [ ]:
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=train_transform)
  test_data = datasets.ImageFolder(test_dir, transform=test_transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names



In [ ]:
BATCH_SIZE=32
augmented_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])
train_dataloader_20_percent_aug, test_dataloader, class_names = create_dataloaders(train_dir=train_dir_20_percent,
                                                                                   test_dir=test_dir,
                                                                                   train_transform=augmented_transform,
                                                                                   test_transform=simple_transform,
                                                                                   batch_size=BATCH_SIZE)

In [ ]:
set_seeds()
effnetb2_data_20_percent_10_epochs_augmented = create_effnetb2()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=effnetb2_data_20_percent_10_epochs_augmented.parameters(),lr=0.001)

train(model=effnetb2_data_20_percent_10_epochs_augmented,
      train_dataloader=train_dataloader_20_percent_aug,
      test_dataloader=test_dataloader,
      optimizer=optimizer,
      loss_fn=loss_fn,
      epochs=10,
      device=device,
      writer=create_writer(experiment_name="data_20_percent_augmented",
                           model_name="effnetb2",extra="10_epochs"))

In [ ]:
%tensorboard --logdir runs

In [ ]:
correct_food101 = Path("data/food101_correct")
train_data = datasets.Food101(root=correct_food101,
                              split="train",
                              # transform=transforms.ToTensor(),
                              download=True)

# Get testing data
test_data = datasets.Food101(root=correct_food101,
                             split="test",
                             # transform=transforms.ToTensor(),
                             download=True)